# NILMTK Rapid Experimentation API

### This notebook demonstrates the use of NILMTK's ExperimentAPI - a  new  NILMTK  interface which  allows NILMTK users to focus on which experiments to run rather than on the code required to run such experiments.

It is important to note that handing over so much flexibility to the user does require the user to be somewhat familiar with the data set, but this part of the process is supported by NILMTK as data exploration is simple and well documented. 

Lets us start with a very simple experiment to demonstrate the use of the API for multiple appliances in a minimal use case. This experiment shows how the user can select the appliances in the dataset on which disaggregation is to be performed.

Importing the API. 

In [1]:
from nilmtk.api import API
import warnings
warnings.filterwarnings("ignore")

Next, we import the required algorithms on which we wish to run the experiments

In [2]:
from nilmtk.disaggregate import CO

Next, we enter the values for the different parameters in the dictionary. Since we need multiple appliances, we enter the names of all the required appliances in the _'appliances'_ parameter.

In [3]:
experiment1 = {
  'power': {'mains': ['apparent','active'],'appliance': ['apparent','active']},
  'sample_rate': 60,
  'appliances': ['air conditioner'],
  'methods': {"CO":CO({})},
  'train': {    
    'datasets': {
        'Dataport': {
            'path': '/app/nilmtk-contrib/data/ukdale.h5',
            'buildings': {
                1: {
                    'start_time': '2013-04-04',
                    'end_time': '2013-04-06'
                    }
                }                
            }
        }
    },
  'test': {
    'datasets': {
        'Dataport': {
            'path': '/app/nilmtk-contrib/data/ukdale.h5',
            'buildings': {
                1: {
                    'start_time': '2013-04-25',
                    'end_time': '2013-04-26'
                    }
                }
            }
        },
        'metrics':['rmse']
    }
}

In this example experimental setup, we have set the _sample rate_ at 60Hz and use Combinatorial Optimisation to 
disaggregate the required appliances from building 10 in the dataport dataset with the _RMSE_ metric to measure the accuracy. We also specify the dates for training and testing

Next we provide this experiment dictionary as input to the API.

In [4]:
api_results_experiment_1 = API(experiment1)

Joint Testing for all algorithms
Loading data for  Dataport  dataset
Dropping missing values
Generating predictions for : CO
...............CO disaggregate_chunk running.............
Estimating power demand for 'microwave'

ValueError: Length of values (2880) does not match length of index (1440)

We can observe the prediction vs. truth graphs in the above cell. The accuracy metrics can be accessed using the following commands:

In [ ]:
errors_keys = api_results_experiment_1.errors_keys
errors = api_results_experiment_1.errors
for i in range(len(errors)):
    print (errors_keys[i])
    print (errors[i])
    print ("\n\n")

This was a trivial experiment that only scratches the surface of the true potential of this API.

In the next experiment we will run an incrementally more complex version of the above experiment. Here we will use multiple models to disaggregate the appliance readings with the models having their own sets of parameters which can be set by the users within the experimental dictionary in order to fine tune experiments.

We also import the required algorithms for the next experiments

In [ ]:
from nilmtk.disaggregate import FHMM_EXACT, Mean

In [ ]:
experiment2 = {
  'power': {'mains': ['apparent','active'],'appliance': ['apparent','active']},
  'sample_rate': 60,
  'appliances': ['fridge','air conditioner', 'microwave'],
  'methods': {"Mean":Mean({}),"FHMM_EXACT":FHMM_EXACT({'num_of_states':2}), "CombinatorialOptimisation":CO({})},
  'train': {    
    'datasets': {
        'Dataport': {
            'path': 'data/dataport.hdf5',
            'buildings': {
                10: {
                    'start_time': '2015-04-04',
                    'end_time': '2015-04-06'
                    }
                }                
            }
        }
    },
  'test': {
    'datasets': {
        'Datport': {
            'path': 'data/dataport.hdf5',
            'buildings': {
                10: {
                    'start_time': '2015-04-25',
                    'end_time': '2015-04-26'
                    }
                }
            }
        },
        'metrics':['mae', 'rmse']
    }
}

In [ ]:
api_results_experiment_2 = API(experiment2)

In [ ]:
api_results_experiment_2.errors

In [ ]:
errors_keys = api_results_experiment_2.errors_keys
errors = api_results_experiment_2.errors
for i in range(len(errors)):
    print (errors_keys[i])
    print (errors[i])
    print ("\n\n")

The API makes running experiments extremely quick and efficient, with the emphasis on creating finely tuned reproducible experiments where model and parameter performances can be easily evaluated at a glance.   

In the next iteration of this experiment, we introduce more parameters _chunksize_, _DROP_ALL_NANS_ and _artificial_aggregate_ and add another disaggregation algorithm (_Hart85_). We also train and test data from multiple buildings of the same dataset.

We also import the Hart algorithm for the next experiment

In [ ]:
from nilmtk.disaggregate import Hart85

In [ ]:
experiment3 = {
  'power': {'mains': ['apparent','active'],'appliance': ['apparent','active']},
  'sample_rate': 60,
  'appliances': ['fridge','air conditioner','electric furnace','washing machine'],
  'artificial_aggregate': True,
  'chunksize': 20000,
  'DROP_ALL_NANS': False,
  'methods': {"Mean":Mean({}),"Hart85":Hart85({}), "FHMM_EXACT":FHMM_EXACT({'num_of_states':2}), "CO":CO({})},
  'train': {    
    'datasets': {
      'Dataport': {
        'path': 'data/dataport.hdf5',
        'buildings': {
          54: {
            'start_time': '2015-01-28',
            'end_time': '2015-02-12'
          },
          56: {
            'start_time': '2015-01-28',
            'end_time': '2015-02-12'
          },
          57: {
            'start_time': '2015-04-30',
            'end_time': '2015-05-14'
          },
                }                
            }
        }
    },
  'test': {
    'datasets': {
      'Datport': {
        'path': 'data/dataport.hdf5',
        'buildings': {
          94: {
            'start_time': '2015-04-30',
            'end_time': '2015-05-07'
          },
          103: {
            'start_time': '2014-01-26',
            'end_time': '2014-02-03'
          },
          113: {
            'start_time': '2015-04-30',
            'end_time': '2015-05-07'
          },
        }
      }
    },
        'metrics':['mae', 'rmse']
    }
}

In [ ]:
api_results_experiment_3 = API(experiment3)

In [ ]:
errors_keys = api_results_experiment_3.errors_keys
errors = api_results_experiment_3.errors
for i in range(len(errors)):
    print (errors_keys[i])
    print (errors[i])
    print ("\n\n")

The results of the above experiment are presented for every chunk per building in the test set.

In the following experiment, we demonstrate how to run experiments across datasets, which was previously not possible. The important thing to pay attention to is that such datasets can only be trained and tested together as long as they have common appliances in homes with common _ac_types_.

In [ ]:
experiment4 = {
  'power': {'mains': ['apparent','active'],'appliance': ['apparent','active']},
  'sample_rate': 60,
  'appliances': ['washing machine','fridge'],
  'artificial_aggregate': True,
  'chunksize': 20000,
  'DROP_ALL_NANS': False,
  'methods': {"Mean":Mean({}),"Hart85":Hart85({}), "FHMM_EXACT":FHMM_EXACT({'num_of_states':2}), 'CO':CO({})},   
  'train': {
    'datasets': {
      'UKDALE': {
        'path': 'C:/Users/Hp/Desktop/nilmtk-contrib/ukdale.h5',
        'buildings': {
              1: {
                'start_time': '2017-01-05',
                'end_time': '2017-03-05'
              },          
            }
          },        
        }
      },    
  'test': {
    'datasets': {
      'DRED': {
        'path': 'C:/Users/Hp/Desktop/nilmtk-contrib/dred.h5',
        'buildings': {
              1: {
                    'start_time': '2015-09-21',
                    'end_time': '2015-10-01'
          }
        }
      },
      'REDD': {
        'path': 'C:/Users/Hp/Desktop/nilmtk-contrib/redd.h5',
        'buildings': {
              1: {
                    'start_time': '2011-04-17',
                    'end_time': '2011-04-27'
          }
        }
      }
    },
        'metrics':['mae', 'rmse']
    }
}

In [ ]:
api_results_experiment_4 = API(experiment4)

In [ ]:
errors_keys = api_results_experiment_4.errors_keys
errors = api_results_experiment_4.errors
for i in range(len(errors)):
    print (errors_keys[i])
    print (errors[i])
    print ("\n\n")

Just like the above experiments, any user can set up other experiments very quickly. 